In [1]:
import glob
import itertools
import os
import yaml
from copy import deepcopy

import numpy as np


In [2]:
# CONFIG_FNAME = "standard_bnfix_adam_beta1_5_spotless_sc.yml"
MODEL_NAME = "ADDA"
CONFIG_DIR = "configs/generated_dlpfc"
CONFIG_FNAME_PREFIX = "gen_dlpfc_dlpfc"

In [3]:
config = {
    "lib_params": {},
    "data_params": {},
    "train_params": {},
    "model_params": {},
}

config["data_params"]["all_genes"] = False
config["data_params"]["data_dir"] = "data"
config["data_params"]["dset"] = "dlpfc"
config["data_params"]["samp_split"] = True
config["data_params"]["sc_id"] = "GSE144136"
config["data_params"]["scaler_name"] = "standard"
config["data_params"]["st_id"] = "spatialLIBD"
config["data_params"]["st_split"] = False

config["model_params"]["adda_kwargs"] = {}

config["train_params"]["epochs"] = 200
config["train_params"]["initial_train_epochs"] = 200
config["train_params"]["adam_beta1"] = 0.5
config["train_params"]["reverse_val"] = False
config["train_params"]["pretraining"] = True

if not os.path.exists(os.path.join(CONFIG_DIR, MODEL_NAME)):
    os.makedirs(os.path.join(CONFIG_DIR, MODEL_NAME))

with open(os.path.join(CONFIG_DIR, MODEL_NAME, f"{CONFIG_FNAME_PREFIX}.yml"), "w") as f:
    yaml.safe_dump(config, f)

In [4]:
## ADDA

# data_params
data_params_lists = dict(
    n_markers=[20, 40, 80],
    n_mix=[3, 5, 8],
    n_spots=[5000, 10000, 20000, 50000],
)

# model_params
model_params_lists = dict(
    bn_momentum=[0.1, 0.01],
    emb_dim=[32, 64],
)
# train_params
train_params_lists = dict(
    alpha=[1, 2, 5],
    batch_size=[512, 1024, 2048],
    dis_loop_factor=[2, 5, 10],
    enc_lr=[0.000002, 0.00002, 0.0002],
    initial_train_lr=[0.0001, 0.0002, 0.001],
)

In [5]:
total_configs = 1
for value in data_params_lists.values():
    total_configs *= len(value)
for value in model_params_lists.values():
    total_configs *= len(value)
for value in train_params_lists.values():
    total_configs *= len(value)
total_configs

34992

In [6]:
config

{'lib_params': {},
 'data_params': {'all_genes': False,
  'data_dir': 'data',
  'dset': 'dlpfc',
  'samp_split': True,
  'sc_id': 'GSE144136',
  'scaler_name': 'standard',
  'st_id': 'spatialLIBD',
  'st_split': False},
 'train_params': {'epochs': 200,
  'initial_train_epochs': 200,
  'adam_beta1': 0.5,
  'reverse_val': False,
  'pretraining': True},
 'model_params': {'adda_kwargs': {}}}

In [7]:
rng = np.random.default_rng(397569)

yes_samples = set(rng.choice(total_configs, size=1000, replace=False))


data_params_l = []
for kv_tuples in itertools.product(
    *[[(k, v) for v in vlist] for k, vlist in data_params_lists.items()]
):
    data_params_l.append(dict(kv_tuples))

model_params_l = []
for kv_tuples in itertools.product(
    *[[(k, v) for v in vlist] for k, vlist in model_params_lists.items()]
):
    model_params_l.append(dict(kv_tuples))

train_params_l = []
for kv_tuples in itertools.product(
    *[[(k, v) for v in vlist] for k, vlist in train_params_lists.items()]
):
    train_params_l.append(dict(kv_tuples))

for i, (data_params, model_params, train_params) in enumerate(
    itertools.product(data_params_l, model_params_l, train_params_l)
):
    if i not in yes_samples:
        continue

    new_config = deepcopy(config)
    
    new_config["data_params"].update(data_params)
    new_config["model_params"]["adda_kwargs"].update(model_params)
    new_config["train_params"].update(train_params)

    new_config["lib_params"]["manual_seed"] = int(rng.integers(0, 2**32))

    version = f"{CONFIG_FNAME_PREFIX}-{i}"
    new_config["model_params"]["model_version"] = version

    with open(os.path.join(CONFIG_DIR, MODEL_NAME, f"{version}.yml"), "w") as f:
        yaml.safe_dump(new_config, f)

In [8]:
print(yaml.safe_dump(new_config))

data_params:
  all_genes: false
  data_dir: data
  dset: dlpfc
  n_markers: 80
  n_mix: 8
  n_spots: 50000
  samp_split: true
  sc_id: GSE144136
  scaler_name: standard
  st_id: spatialLIBD
  st_split: false
lib_params:
  manual_seed: 1470734752
model_params:
  adda_kwargs:
    bn_momentum: 0.01
    emb_dim: 64
  model_version: gen_dlpfc_dlpfc-34855
train_params:
  adam_beta1: 0.5
  alpha: 2
  batch_size: 512
  dis_loop_factor: 10
  enc_lr: 0.0002
  epochs: 200
  initial_train_epochs: 200
  initial_train_lr: 0.0002
  pretraining: true
  reverse_val: false



In [9]:
lines = [
    os.path.basename(name)
    for name in sorted(glob.glob(os.path.join(CONFIG_DIR, MODEL_NAME, f"{CONFIG_FNAME_PREFIX}-*.yml")))
]
with open(
    os.path.join(CONFIG_DIR, MODEL_NAME, "a_list.txt"),
    mode="wt",
    encoding="utf-8",
) as myfile:
    myfile.write("\n".join(lines))
    myfile.write("\n")